In [1]:
import matplotlib_inline
import pandas as pd
import matplotlib.pyplot as plt

pd.options.mode.copy_on_write = True
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

## 讀取資料

In [2]:
df = pd.read_excel("../Online_Retail001.xlsx")
df.shape

(10000, 8)

df.head()

In [3]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,10000.000000,10000,10000.000000,7526.000000
mean,10.158300,2011-07-02 06:06:49.842000128,3.881760,15266.947250
min,-600.000000,2010-12-01 08:45:00,0.000000,12347.000000
25%,1.000000,2011-03-24 13:10:00,1.250000,13908.000000
50%,3.000000,2011-07-15 14:05:00,2.080000,15128.000000
75%,12.000000,2011-10-17 15:06:15,4.130000,16759.000000
max,4000.000000,2011-12-09 12:50:00,736.380000,18283.000000
std,57.905129,NaN,14.863321,1704.574949


發現 `Quantity` 有負值，`CustomerID` 有空值

## 資料清理

### 處理 `Quantity` 負值

In [4]:
df = df[df["Quantity"] > 0]

### 處理 `CustomerID` 空值問題

計算空值個數。

In [5]:
df["CustomerID"].isnull().sum()

np.int64(2442)

刪除 customerid 中的空值。

In [6]:
df.dropna(subset=["CustomerID"], inplace=True)

In [7]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,2010-12-01 08:45:00,3.75,12583.0,France
1,536375,71053,WHITE METAL LANTERN,6,2010-12-01 09:32:00,3.39,17850.0,United Kingdom
2,536381,21934,SKULL SHOULDER BAG,10,2010-12-01 09:41:00,1.65,15311.0,United Kingdom
3,536381,47580,TEA TIME DES TEA COSY,2,2010-12-01 09:41:00,2.55,15311.0,United Kingdom
4,536384,22424,ENAMEL BREAD BIN CREAM,8,2010-12-01 09:53:00,10.95,18074.0,United Kingdom


### 計算 Sales

In [8]:
df["Sales"] = df["Quantity"] * df["UnitPrice"]

In [9]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,2010-12-01 08:45:00,3.75,12583.0,France,45.00
1,536375,71053,WHITE METAL LANTERN,6,2010-12-01 09:32:00,3.39,17850.0,United Kingdom,20.34
2,536381,21934,SKULL SHOULDER BAG,10,2010-12-01 09:41:00,1.65,15311.0,United Kingdom,16.50
3,536381,47580,TEA TIME DES TEA COSY,2,2010-12-01 09:41:00,2.55,15311.0,United Kingdom,5.10
4,536384,22424,ENAMEL BREAD BIN CREAM,8,2010-12-01 09:53:00,10.95,18074.0,United Kingdom,87.60


## 資料分析

## 計算每個顧客的 RFM 資料

抓出 InvoiceDate 的最大值。

In [10]:
maximum = df["InvoiceDate"].max()

maximum

Timestamp('2011-12-09 12:50:00')

為了簡化接下來的操作，假設今天是 2011-12-31。

In [11]:
from datetime import datetime


today = datetime(2011, 12, 31)

計算每個客戶的 RFM。

In [12]:
summary_df = df.groupby("CustomerID").agg(
    Recency=("InvoiceDate", lambda x: (today - x.max())),
    Frequency=("InvoiceNo", "nunique"),
    Monetary=("Sales", "sum"),
)

summary_df

,Recency,Frequency,Monetary
CustomerID,,,
12347.0,23 days 08:08:00,2,28.20
12348.0,96 days 10:47:00,1,40.00
12349.0,39 days 14:09:00,1,339.00
12352.0,93 days 09:02:00,2,36.60
12354.0,253 days 10:49:00,1,23.40
...,...,...,...
18261.0,116 days 14:26:00,1,17.00
18263.0,46 days 15:00:00,2,40.32
18265.0,93 days 09:50:00,1,15.90


In [13]:
summary_df["Recency"].dtype

dtype('<m8[ns]')

把 Recency 轉成天數。

In [14]:
summary_df["RecencyDays"] = summary_df["Recency"].dt.days

summary_df

,Recency,Frequency,Monetary,RecencyDays
CustomerID,,,,
12347.0,23 days 08:08:00,2,28.20,23
12348.0,96 days 10:47:00,1,40.00,96
12349.0,39 days 14:09:00,1,339.00,39
12352.0,93 days 09:02:00,2,36.60,93
12354.0,253 days 10:49:00,1,23.40,253
...,...,...,...,...
18261.0,116 days 14:26:00,1,17.00,116
18263.0,46 days 15:00:00,2,40.32,46
18265.0,93 days 09:50:00,1,15.90,93


## RFM 分類，切四等份

In [15]:
summary_df["RecencyDays"] = summary_df["RecencyDays"]
quantiles = summary_df.quantile(q=[0.25, 0.5, 0.75])

quantiles

,Recency,Frequency,Monetary,RecencyDays
0.25,49 days 09:04:00,1.0,13.5075,49.00
0.50,94 days 14:49:30,1.0,26.1000,94.00
0.75,211 days 19:15:15,2.0,56.4625,211.25


### 定義 Recency 的分數

- 第一四分位以內的天數資料，給予4分
- 第三四分位數後的資料給予1分。

In [16]:
def R_class(x: int, p: str, d: pd.DataFrame) -> int:
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1

### 執行 RFM 分類

In [17]:
RFM_Segment = summary_df.copy()

RFM_Segment["r_quartile"] = RFM_Segment["RecencyDays"].apply(
    R_class, args=("RecencyDays", quantiles)
)

RFM_Segment

,Recency,Frequency,Monetary,RecencyDays,r_quartile
CustomerID,,,,,
12347.0,23 days 08:08:00,2,28.20,23,4
12348.0,96 days 10:47:00,1,40.00,96,2
12349.0,39 days 14:09:00,1,339.00,39,4
12352.0,93 days 09:02:00,2,36.60,93,3
12354.0,253 days 10:49:00,1,23.40,253,1
...,...,...,...,...,...
18261.0,116 days 14:26:00,1,17.00,116,2
18263.0,46 days 15:00:00,2,40.32,46,4
18265.0,93 days 09:50:00,1,15.90,93,3


## 練習：`FM_Class(x,p,d)` 怎麼寫？

- 定義 `Frequency`、`Monetary` 的分數
- 第三四分位以後的天數資料，給予 4 分
- 第一四分位數內的資料給予 1 分

In [18]:
def FM_class(x: int, p: str, d: pd.DataFrame) -> int:
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

In [19]:
RFM_Segment["F_Quartile"] = RFM_Segment["Frequency"].apply(
    FM_class, args=("Frequency", quantiles)
)
RFM_Segment["M_Quartile"] = RFM_Segment["Monetary"].apply(
    FM_class, args=("Monetary", quantiles)
)

RFM_Segment

,Recency,Frequency,Monetary,RecencyDays,r_quartile,F_Quartile,M_Quartile
CustomerID,,,,,,,
12347.0,23 days 08:08:00,2,28.20,23,4,3,3
12348.0,96 days 10:47:00,1,40.00,96,2,1,3
12349.0,39 days 14:09:00,1,339.00,39,4,1,4
12352.0,93 days 09:02:00,2,36.60,93,3,3,3
12354.0,253 days 10:49:00,1,23.40,253,1,1,2
...,...,...,...,...,...,...,...
18261.0,116 days 14:26:00,1,17.00,116,2,1,2
18263.0,46 days 15:00:00,2,40.32,46,4,3,3
18265.0,93 days 09:50:00,1,15.90,93,3,1,2


## 編號

將 RFM 三欄的分數變成字串，形成新的欄位。

對於賣場最有價值的顧客其分類代號為「444」，最沒有價值的顧客其分類代號為「111」。

In [20]:
RFM_Segment["RFMClass"] = (
    RFM_Segment.r_quartile.map(str)
    + RFM_Segment.F_Quartile.map(str)
    + RFM_Segment.M_Quartile.map(str)
)

# 其他做法：
# RFM_Segment['RFMClass'] = RFM_Segment[['r_quartile', 'F_Quartile', 'M_Quartile']].astype(str).agg(''.join, axis=1)
# RFM_Segment['RFMClass'] = RFM_Segment.apply(lambda row: f"{row['r_quartile']}{row['F_Quartile']}{row['M_Quartile']}", axis=1)
# RFM_Segment['RFMClass'] = RFM_Segment['r_quartile'].astype(str) + RFM_Segment['F_Quartile'].astype(str) + RFM_Segment['M_Quartile'].astype(str)

RFM_Segment

,Recency,Frequency,Monetary,RecencyDays,r_quartile,F_Quartile,M_Quartile,RFMClass
CustomerID,,,,,,,,
12347.0,23 days 08:08:00,2,28.20,23,4,3,3,433
12348.0,96 days 10:47:00,1,40.00,96,2,1,3,213
12349.0,39 days 14:09:00,1,339.00,39,4,1,4,414
12352.0,93 days 09:02:00,2,36.60,93,3,3,3,333
12354.0,253 days 10:49:00,1,23.40,253,1,1,2,112
...,...,...,...,...,...,...,...,...
18261.0,116 days 14:26:00,1,17.00,116,2,1,2,212
18263.0,46 days 15:00:00,2,40.32,46,4,3,3,433
18265.0,93 days 09:50:00,1,15.90,93,3,1,2,312


### 問題

顧客分類代號為 “444” 的黃金顧客佔全部顧客的比例                       

In [21]:
print(
    (RFM_Segment[RFM_Segment["RFMClass"] == "444"].shape[0] / RFM_Segment.shape[0])
    * 100,
    "%",
)

7.89695945945946 %
